### Data Re-uploading quantum Circuit

Data re-uploading quantum circuit, a technique used in quantum machine learning where classical data is repeatedly encoded into a quantum circuit through multiple layers of operations.


## Purpose: This circuit encodes a classical data point (x) multiple times into a single qubit circuit using trainable parameters (weights).

## Key Idea:

The input data x is re-injected into the circuit in each layer via rotation gates.

The weights are trainable parameters that control how the data is transformed.

## Structure:

Uses RY (Y-rotation) and RZ (Z-rotation) gates.

The final expectation value of PauliZ is measured.

## Inputs
1. x (float):

- A single classical data point (e.g., 0.5).

- This is the input feature being processed.

2. weights (numpy array):

- A trainable parameter matrix of shape (num_layers, 2).

- Example: weights = np.random.rand(3, 2) → 3 layers, 2 parameters per layer.

## Output
expval(qml.PauliZ(0)) (float):

- The expectation value of the Pauli-Z operator on the first qubit.

- Returns a value between -1 and 1 (since PauliZ has eigenvalues ±1).



# How does it work?

1. What is qml.expval(qml.PauliZ(0))?

Measures the average value of the Pauli-Z operator on the qubit.

Possible outcomes:

+1 if the qubit is in state |0⟩.

-1 if the qubit is in state |1⟩.

A value between -1 and 1 if the qubit is in a superposition or rotated state.



### The output depends on: 

Input data (x = 0.5): Scaled by weights in each RY rotation.

Random weights (e.g., weights = np.random.rand(3, 2)): Each layer applies a small rotation, cumulatively affecting the final state.

Final state before measurement:

The sequence of RY and RZ rotations brings the qubit to a state like:
ψ⟩=cos(θ/2)∣0⟩+e^(iϕ)sin(θ/2)∣1⟩ where θ and ϕ depend on the weights and x.


Each time you run the code, weights = np.random.rand(3, 2) generates new random values for the weights (uniformly sampled between 0 and 1).

Since the output (PauliZ expectation value) depends entirely on these weights, different weights produce different outputs.

Example:

If weights = [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]] → Output might be 0.47.

If weights = [[0.9, 0.8], [0.7, 0.6], [0.5, 0.4]] → Output could be 0.91.



our circuit applies sequential rotations (RY and RZ) controlled by x * weights[i, 0] and weights[i, 1].

Small changes in weights lead to large changes in the final quantum state due to:

Nonlinearity of quantum rotations: Trigonometric functions (sin, cos) in rotation gates amplify small weight differences.

Cumulative effect: 3 layers of rotations compound the impact of randomness.




The output is qml.expval(qml.PauliZ(0)), which measures the qubit's alignment with the |0⟩ state.

The expectation value is calculated as:


⟨Z⟩=cos(θtotal)
where θtotal is the net rotation angle from all layers.

Because cos(θ) is highly sensitive to θ, small weight variations produce vastly different outputs.


So, the output varies over time and won't be same all the time unles......... (we fix the random seed)


### Key Takeaways
The variability is expected and stems from:

Random weight initialization (np.random.rand).

Sensitivity of quantum rotations to small parameter changes.

To get consistent results:

Fix the random seed (e.g., np.random.seed(42)).

Train the weights for a specific task.


In [29]:
import pennylane as qml
import numpy as np
from pennylane import numpy as np
dev = qml.device("default.qubit", wires=1)

def layer(x, weights, i):
    qml.RY(x * weights[i,0], wires=0)
    qml.RZ(weights[i,1], wires=0)

@qml.qnode(dev)
def data_reuploading(x, weights):
    for i in range(weights.shape[0]):
        layer(x, weights, i)
    return qml.expval(qml.PauliZ(0))

data = 0.5
weights = np.random.rand(3,2)  # 3 layers, 2 params each
print(data_reuploading(data, weights))

0.9183380968138689
